<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">Colab KSO Tutorials #3: Upload clips to Zooniverse</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Mar 30, 2022</h5>

# Set up and requirements

In [ ]:
# Code to try fix the upload video panoptes libmagic
# !pip install panoptes-client
# !pip uninstall python-magic
# !pip install python-magic==0.4.25
# !pip install python-magic-bin
# !pip install libmagic==1.0
# !pip uninstall python-magic
# !pip install python-libmagic
# !pip install git+https://github.com/julian-r/python-magic.git

In [ ]:
!git clone --recurse-submodules -b dev https://github.com/ocean-data-factory-sweden/koster_data_management.git
!pip install -r koster_data_management/requirements.txt

In [ ]:
# Test if panoptes can be loaded
try:
  !pip install git+https://github.com/zooniverse/panoptes-python-client.git
  import panoptes_client
except:
  print('Restarting runtime...')
  exit()

### Import Python packages

In [1]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

os.chdir("koster_data_management/tutorials")
sys.path.append('..')


# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.movie_utils as m_utils
import kso_utils.t3_utils as t3
import kso_utils.project_utils as p_utils

print("Packages loaded successfully")

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2416640/45929032 bytes (5.3%)5955584/45929032 bytes (13.0%)9166848/45929032 bytes (20.0%)12206080/45929032 bytes (26.6%)15736832/45929032 bytes (34.3%)19128320/45929032 bytes (41.6%)22667264/45929032 bytes (49.4%)26132480/45929032 bytes (56.9%)29024256/45929032 bytes (63.2%)31866880/45929032 bytes (69.4%)33923072/45929032 bytes (73.9%)36814848/45929032 bytes (80.2%)

### Choose your project

In [2]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU', 'Medins'), value='…

In [3]:
project = p_utils.find_project(project_name=project_name.value)

### Initiate sql and zoo project

In [4]:
# Initiate db
db_info_dict = t_utils.initiate_db(project)

Enter the key id for the aws server··········
Enter the secret access key for the aws server··········


None/sites_buv_doc.csv: 100%|██████████| 145k/145k [00:00<00:00, 146kB/s]
None/movies_buv_doc.csv: 100%|██████████| 14.2k/14.2k [00:00<00:00, 32.9kB/s]
None/species_buv_doc.csv: 100%|██████████| 7.53k/7.53k [00:00<00:00, 17.4kB/s]
None/surveys_buv_doc.csv: 100%|██████████| 2.25k/2.25k [00:00<00:00, 5.38kB/s]
None/choices_buv.csv: 100%|██████████| 3.54k/3.54k [00:00<00:00, 8.03kB/s]


Updated sites
Updated movies
Updated species


In [5]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project)

Enter your Zooniverse user··········
Enter your Zooniverse password··········


## Retrieve info about zooniverse clips

In [6]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(project = project, 
                                                    db_info_dict = db_info_dict,
                                                    zoo_project = zoo_project,
                                                    zoo_info = ["subjects"])

Retrieving subjects from Zooniverse
subjects were retrieved successfully
Updated subjects
The database has a total of 978 frame subjects and 4993 clip subjects have been updated


## Retrieve info about movies hosted in the server

In [7]:
# Store info about the movies available in the server
available_movies_df = t3.retrieve_movie_info_from_server(project = project,
                                                         db_info_dict = db_info_dict)

# Create some clip examples

### Select the movie you want to upload to Zooniverse

In [ ]:
movie_i = t3.movie_to_upload(available_movies_df = available_movies_df)

### Check if movie is already in Zooniverse

Remember to query the newest zooniverse data to get the most up to date list of clips uploaded

In [ ]:
# Check movie hasn't been uploaded to Zooniverse
t3.check_movie_uploaded(movie_i = movie_i.value,
                        db_info_dict = db_info_dict)

## Have a quick look at the movie selected

In [ ]:
t3.preview_movie(project,db_info_dict,available_movies_df,movie_i)

## Create a few clip examples

Test different parameters (e.g. compression rate, color modifications) in randomly selected clip examples

In [ ]:
random_clips_info = t3.select_random_clips(movie_i = movie_i.value,
                                           db_info_dict = db_info_dict)

In [ ]:
temp_clips = t3.create_clips(available_movies_df = available_movies_df,
                             movie_i = movie_i.value,
                             db_info_dict = db_info_dict,
                             clip_selection = random_clips_info,  
                             project = project,
                             example_clips = True)

## Review the size of the clip examples

In [ ]:
t3.check_clip_size(clips_df = temp_clips)

## Select the clip modification

In [ ]:
clip_modification = t3.clip_modification_widget()
clip_modification

## Modify the clip examples

### Modify the clips WITH GPU

#### Install the requirements for GPU video modification

In [ ]:
# Install ffmpeg with GPU version
!git clone https://github.com/rokibulislaam/colab-ffmpeg-cuda.git
!cp -r ./colab-ffmpeg-cuda/bin/. /usr/bin/

#### Modify the clips with GPU

In [ ]:
clips_ready_df = t3.modify_clips(clips_to_upload_df = temp_clips.reset_index(drop=True),
                              movie_i = movie_i.value,
                              modification_details = clip_modification.checks,
                              gpu_available = True,
                              project = project
                              )

### Modify the clips WITHOUT GPU

In [ ]:
clip_examples_df = t3.modify_clips(clips_to_upload_df = temp_clips.reset_index(drop=True),
                              movie_i = movie_i.value,
                              modification_details = clip_modification.checks,
                              gpu_available = True,
                              project = project
                              )

## Review the size of the modified clips

In [ ]:
t3.check_clip_size(clips_df = clips_ready_df)

## Compare the clips

In [ ]:
t3.compare_clips(df = clips_ready_df)

# Create the clips to upload to Zooniverse

## Have a quick look at the movie selected

In [ ]:
t3.preview_movie(project,db_info_dict,available_movies_df,movie_i)

## Specify the number of clips and clip length

In [ ]:
clip_selection = t3.select_clip_n_len(movie_i = movie_i.value,
                                      db_info_dict = db_info_dict)

## Review the clips that will be created

In [ ]:
t3.review_clip_selection(clip_selection = clip_selection, 
                         movie_i = movie_i.value)

## Create the clips

In [ ]:
clips_to_upload_df = t3.create_clips(available_movies_df = available_movies_df, 
                                     movie_i = movie_i.value,
                                     db_info_dict = db_info_dict,
                                     clip_selection = clip_selection,  
                                     project = project)

## Review the size of the clips

In [ ]:
t3.check_clip_size(clips_df = clips_to_upload_df)

## Video modifications

### Modify the clips WITH GPU

In [ ]:
clips_ready_df = t3.modify_clips(clips_to_upload_df = clips_to_upload_df.reset_index(drop=True),
                              movie_i = movie_i.value,
                              modification_details = clip_modification.checks,
                              gpu_available = True,
                              project = project
                              )

### Modify the clips WITHOUT GPU

In [ ]:
clips_ready_df = t3.modify_clips(clips_to_upload_df = clips_to_upload_df.reset_index(drop=True), 
                                 movie_i = movie_i.value,
                                 modification_details = clip_modification.checks,
                                 project = project)

## Review the size of the modified clips

In [ ]:
t3.check_clip_size(clips_df = clips_ready_df)

## Compare the clips

In [ ]:
t3.compare_clips(df = clips_ready_df)

# Upload clips to Zooniverse

## Set Zooniverse metadata

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [ ]:
upload_to_zoo, sitename, created_on = t3.set_zoo_metadata(db_info_dict = db_info_dict, 
                                                          df = clips_ready_df,
                                                          project = project)

## Upload clips to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [ ]:
t3.upload_clips_to_zooniverse(upload_to_zoo = upload_to_zoo, 
                              sitename = sitename,
                              created_on = created_on,
                              project = project.Zooniverse_number)

In [ ]:
#END